In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from statistics import median

In [2]:
## Reading the Amazon and Flipkart file ##

In [3]:
amzn = pd.read_csv("amzn.csv", encoding= 'unicode_escape')
flpkt = pd.read_csv("flpkt.csv", encoding= 'unicode_escape')

In [4]:
amzn.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
flpkt.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [6]:
amzn.shape, flpkt.shape

((20000, 15), (20000, 15))

In [7]:
## Data Cleaning ##

In [8]:
# Removing products form data with negative retail price or negative discounted price
amzn[amzn.retail_price < 0].shape[0], amzn[amzn.discounted_price < 0].shape[0]

(78, 0)

In [9]:
flpkt[flpkt.retail_price < 0].shape[0], flpkt[flpkt.retail_price < 0].shape[0]

(0, 0)

In [10]:
amzn = amzn[(amzn.retail_price >= 0) & (amzn.discounted_price >= 0)].reset_index(drop=True)

In [11]:
# There are products with Discounted Price higher than Retail Price, but we will not remove it
# Because it doesn't harm our calculation and can be removed in the end too.
amzn[amzn.retail_price < amzn.discounted_price].shape[0]

4247

In [12]:
flpkt[flpkt.retail_price < flpkt.discounted_price].shape[0]

0

In [13]:
# amzn = amzn[amzn.retail_price >= amzn.discounted_price].reset_index(drop=True)

In [14]:
amzn.shape, flpkt.shape

((19922, 15), (20000, 15))

In [15]:
## Finding a common name for same product with different names ##

In [16]:
ps = PorterStemmer()
def edit_name(name):
    edited_name = re.sub('[^a-zA-Z]',' ',name)
    edited_name = edited_name.lower()
    edited_name = edited_name.split()
    edited_name = [ps.stem(word) for word in edited_name if not word in stopwords.words('english')]
    edited_name = ' '.join(edited_name)
    return edited_name

In [17]:
corpus_amzn = []
for i in range(0, len(amzn)):
    edited_name = edit_name(amzn['product_name'][i])
    corpus_amzn.append(edited_name)
corpus_amzn[:10]

['alisha solid women cycl short',
 'fabhomedecor fabric doubl sofa bed',
 'aw belli',
 'alisha solid women cycl short',
 'sicon purpos arnica dog shampoo',
 'etern gandhi super seri crystal paper weight silver finish',
 'alisha solid women cycl short',
 'fabhomedecor fabric doubl sofa bed',
 'dilli bazaaar belli corpor casual casual',
 'alisha solid women cycl short']

In [18]:
corpus_flpkt = []
for i in range(0, len(flpkt)):
    edited_name = edit_name(flpkt['product_name'][i])
    corpus_flpkt.append(edited_name)
corpus_flpkt[:10]

['alisha solid women cycl short',
 'fabhomedecor fabric doubl sofa bed',
 'aw belli',
 'alisha solid women cycl short',
 'sicon purpos arnica dog shampoo',
 'etern gandhi super seri crystal paper weight silver finish',
 'alisha solid women cycl short',
 'fabhomedecor fabric doubl sofa bed',
 'dilli bazaaar belli corpor casual casual',
 'alisha solid women cycl short']

In [19]:
amzn['corpus_amzn'] = corpus_amzn
flpkt['corpus_flpkt'] = corpus_flpkt

In [20]:
## Comparing the price of unique products available on Amazon with prices in Flipkart ##

In [21]:
compare = pd.DataFrame(columns= ['Product name in Amazon', 'Retail Price in Amazon', 'Discounted Price in Amazon',
                                 'Product name in Flipkart', 'Retail Price in Flipkart' , 'Discounted Price in Flipkart'])
for corpus in set(amzn.corpus_amzn):
    location_amzn = np.array(amzn.index[amzn['corpus_amzn']== corpus])
    location_flpkt = np.array(flpkt.index[flpkt['corpus_flpkt']== corpus])
    if len(location_flpkt) != 0:
        new_row = pd.DataFrame({'Product name in Amazon': [amzn.product_name[location_amzn[0]]] ,
                                'Retail Price in Amazon': [median(amzn.retail_price[location_amzn])],
                                'Discounted Price in Amazon': [median(amzn.discounted_price[location_amzn])],
                                'Product name in Flipkart': [flpkt.product_name[location_flpkt[0]]],
                                'Retail Price in Flipkart': [median(flpkt.retail_price[location_flpkt])],
                                'Discounted Price in Flipkart': [median(flpkt.discounted_price[location_flpkt])] })
        compare = pd.concat([compare, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({'Product name in Amazon': [amzn.product_name[location_amzn[0]]] ,
                                'Retail Price in Amazon': [median(amzn.retail_price[location_amzn])],
                                'Discounted Price in Amazon': [median(amzn.discounted_price[location_amzn])],
                                'Product name in Flipkart': 'Product Not Available in Flipkart',
                                'Retail Price in Flipkart': 'NA',
                                'Discounted Price in Flipkart': 'NA' })
        compare = pd.concat([compare, new_row], ignore_index=True)

In [22]:
print("Total no. of unique products available in Amazon and Flipkart:", len(set(amzn.corpus_amzn).union(set(flpkt.corpus_flpkt))))
print("No. of unique product which are available in Amazon:", len(set(corpus_amzn)))
print("No. of unique products which are available in Flipkart but not in Amazon:", len(set(flpkt.corpus_flpkt)-set(amzn.corpus_amzn)))
print( len(set(corpus_amzn)) + len(set(flpkt.corpus_flpkt)-set(amzn.corpus_amzn)), "=", len(set(corpus_amzn)), "+", len(set(flpkt.corpus_flpkt)-set(amzn.corpus_amzn)))
print("Length of our Price Comparison file till now:", compare.shape[0])

Total no. of unique products available in Amazon and Flipkart: 11404
No. of unique product which are available in Amazon: 11339
No. of unique products which are available in Flipkart but not in Amazon: 65
11404 = 11339 + 65
Length of our Price Comparison file till now: 11339


In [23]:
## Comparing the price of unique products which are available on Flipkart but not available on Amazon ##

In [24]:
for corpus in (set(flpkt.corpus_flpkt)-set(amzn.corpus_amzn)):
    location_flpkt = np.array(flpkt.index[flpkt['corpus_flpkt']== corpus])
    new_row = pd.DataFrame({'Product name in Amazon': 'Product Not Available in Amazon' ,
                            'Retail Price in Amazon': 'NA',
                            'Discounted Price in Amazon': 'NA',
                            'Product name in Flipkart': [flpkt.product_name[location_flpkt[0]]],
                            'Retail Price in Flipkart': [median(flpkt.retail_price[location_flpkt])],
                            'Discounted Price in Flipkart': [median(flpkt.discounted_price[location_flpkt])] })
    compare = pd.concat([compare, new_row], ignore_index=True)

In [25]:
## 'compare' dataframe contains the comparison of prices of unique products available on Amazon and Flipkart ##

In [26]:
compare.shape  

(11404, 6)

In [27]:
## compare file has less no. of items than total no. of items avaialble on Flipkart/Amazon because it does not count repetitions.

In [28]:
compare.head()

,Product name in Amazon,Retail Price in Amazon,Discounted Price in Amazon,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart
0,Durian Madison Leatherette 1 Seater Sofa,36592,26853,Durian Madison Leatherette 1 Seater Sofa,36600.0,23790.0
1,YO BABY GIRL'S TROUSERS,841,237,Yo Baby Girl's Trousers,850.0,204.0
2,STREET LONDON Lace Up,1298,1011,STREET LONDON Lace Up,1299.0,799.0
3,Dressberry Casual Sleeveless Solid Women's Top,937.0,504.0,Dressberry Casual Sleeveless Solid Women's Top,949.0,429.0
4,ADA JEWEL ALLOY JEWEL SET,1087,992,Ada Jewel Alloy Jewel Set,1099.0,899.0


In [29]:
compare.to_excel("compare.xlsx")